In [1]:
import os 
import sys

import healpy as hp
import matplotlib.pyplot as plt
import numpy as np

import toast

In [2]:
import scipy.signal as si

In [3]:
import pysm3
import pysm3.units as usm

In [4]:
import skytools

In [5]:
import astropy.units as u 

In [6]:
env = toast.utils.Environment.get()
comm, procs, rank = toast.get_world()
nthread = os.environ["OMP_NUM_THREADS"]

In [7]:
mem = toast.utils.memreport(msg="(whole node)", comm=comm, silent=False)

Memory usage (whole node)
       total :  503.138 GB
   available :  237.310 GB
     percent :   52.800 % 
        used :  190.945 GB
        free :   86.081 GB
      active :   39.231 GB
    inactive :  244.315 GB
     buffers :  108.793 MB
      cached :  226.005 GB
      shared :   58.346 GB
        slab :   97.043 GB



0. Read Planck map

In [8]:
NSIDE = 512
lmax = 2*NSIDE

In [9]:
planck_GC512 = hp.read_map(filename='planck_030_GC_512.FITS',field=None)

Create a telescope and corresponding pointing

1. Start by creating a focalplane with a sample rate and number of detectors

In [10]:
focalplane = toast.instrument.Focalplane(sample_rate=10*u.Hz#,thinfp=256
                                        )
##One fourth of total detectors in the focalplane
## For S4, sample rate ~10-100

In [11]:
with toast.io.H5File("focalplane_SAT3_SAT_f030.h5", "r", comm=comm, force_serial=True) as f:
            focalplane.load_hdf5(f.handle, comm=comm)

Print focalplane information

In [12]:
focalplane

<Focalplane: 252 detectors, sample_rate = 10.0 Hz, FOV = 29.77320077597548 deg, detectors = [029_000_SAT_f030_A .. 040_011_SAT_f030_B]>

Define the schedule, site and telescope

In [13]:
schedule = toast.schedule.GroundSchedule()
schedule.read("POLE_DEEP-169-7.txt", comm=comm)

TOAST INFO: Loading schedule from POLE_DEEP-169-7.txt
TOAST INFO: Loaded 1 scans from POLE_DEEP-169-7.txt totaling 84.0 minutes.


Print schedule information

In [14]:
schedule

<GroundSchedule site=SOUTH_POLE at -89.991 deg, -44.65 deg, 2843.0 m telescope SAT with 1 scans
  <GroundScan 'POLE_DEEP' at 2027-05-28T15:00:00+00:00 with El = 54.1 deg, Az 288.19 deg -- 358.87 deg>
>

Print telescope information

In [15]:
site = toast.instrument.GroundSite(
        schedule.site_name,
        schedule.site_lat,
        schedule.site_lon,
        schedule.site_alt,
        weather=None,
    )
telescope = toast.instrument.Telescope(
        "My_telescope", focalplane=focalplane, site=site
    )

In [16]:
telescope

<Telescope 'My_telescope': uid = 3533178314, site = <GroundSite 'SOUTH_POLE' : uid = 588178429, lon = -44.64999999999998 deg, lat = -89.991 deg, alt = 2842.9999999993684 m, weather = None>, focalplane = <Focalplane: 252 detectors, sample_rate = 10.0 Hz, FOV = 29.77320077597548 deg, detectors = [029_000_SAT_f030_A .. 040_011_SAT_f030_B]>>

We can have multiple groups for parallel computing (start with one). Create a Data Instance and simulate a ground observation operator.

In [17]:
toast_comm = toast.Comm(world=comm, groupsize=1)

In [18]:
data = toast.Data(comm=toast_comm)

In [19]:
sim_ground = toast.ops.SimGround(name="sim_ground", 
                                 weather="south_pole", 
                                 detset_key="pixel", 
                                 telescope = telescope, 
                                 schedule = schedule
                                ) ##simulate motion of the boresight

In [20]:
sim_ground

<SimGround
  API = 0 # Internal interface version for this operator
  azimuth = azimuth # Observation shared key for Azimuth
  boresight_azel = boresight_azel # Observation shared key for boresight AZ/EL
  boresight_radec = boresight_radec # Observation shared key for boresight RA/DEC
  det_data = signal # Observation detdata key to initialize
  det_data_units = K # Output units if creating detector data
  det_flags = flags # Observation detdata key for flags to initialize
  detset_key = pixel # If specified, use this column of the focalplane detector_data to group detectors
  distribute_time = False # Distribute observation data along the time axis rather than detector axis
  el_mod_amplitude = 1.0 deg # Range of elevation modulation
  el_mod_rate = 0.0 Hz # Modulate elevation continuously at this rate
  el_mod_sine = False # Modulate elevation with a sine wave instead of a triangle wave
  el_mod_step = 0.0 deg # Amount to step elevation after each left-right scan pair
  elevation = e

In [21]:
sim_ground.apply(data)

In [22]:
data.obs[0]
## Session associated to weather simulation, not individual optic tubes

<Observation
  name = 'POLE_DEEP-169-7'
  uid = '2008944542'  group has a single process (no MPI)
  telescope = <Telescope 'My_telescope': uid = 3533178314, site = <GroundSite 'SOUTH_POLE' : uid = 588178429, lon = -44.64999999999998 deg, lat = -89.991 deg, alt = 2842.9999999993684 m, weather = <SimWeather : 'south_pole', year = 2027, month = 4, hour = 15, site UID = 588178429, realization = 0, median = False)>, focalplane = <Focalplane: 252 detectors, sample_rate = 10.0 Hz, FOV = 29.77320077597548 deg, detectors = [029_000_SAT_f030_A .. 040_011_SAT_f030_B]>>
  session = <Session 'POLE_DEEP-169-7': uid = 1708650531, start = 2027-05-28 15:00:00+00:00, end = 2027-05-28 16:23:59.900000+00:00>
  scan_el = 54.1 deg
  scan_min_az = -1.2787014485740134 rad
  scan_max_az = 6.263463086632051 rad
  scan_min_el = 0.9442231253289323 rad
  scan_max_el = 0.9442231253289323 rad
  50400 total samples (50400 local)
  shared:  <SharedDataManager
    times (column): shape=(50400,), dtype=float64
    posit

In [23]:
ob = data.obs[0]
ob.detdata.create(name = 'Planck',units = u.K)

In [24]:
pointing = toast.ops.PointingDetectorSimple() ##boresight pointing into detector frame (RA/DEC by default)
weights = toast.ops.StokesWeights(detector_pointing = pointing,mode = "IQU")
pixels = toast.ops.PixelsHealpix(detector_pointing = pointing, nside = NSIDE)

In [25]:
step_0 = 4*u.second


In [26]:
templates = [toast.templates.Offset(name="baselines", step_time = step_0)]

In [27]:
templates

[<Offset
   data = None # This must be an instance of a Data class (or None)
   debug_plots = None # If not None, make debugging plots in this directory
   det_data = signal # Observation detdata key for the timestream data
   det_data_units = K # Desired units of detector data
   det_flag_mask = 7 # Bit mask value for solver flags
   det_flags = flags # Observation detdata key for solver flags to use
   det_mask = 1 # Bit mask value for per-detector flagging
   enabled = True # If True, this class instance is marked as enabled
   good_fraction = 0.5 # Fraction of unflagged samples needed to keep a given offset amplitude
   kernel_implementation = 0 # Which kernel implementation to use (DEFAULT, COMPILED, NUMPY, JAX).
   name = baselines # The 'name' of this class instance
   noise_model = None # Observation key containing the optional noise model
   precond_width = 20 # Preconditioner width in terms of offsets / baselines
   step_time = 4.0 s # Time per baseline step
   times = times 

In [28]:
template_matrix = toast.ops.TemplateMatrix(templates=templates)

In [29]:
scan_map = toast.ops.ScanHealpixMap(name="scan_healpix_map", file='file_c1d0s0_30GHz.FITS',pixel_pointing = pixels, stokes_weights = weights)
scan_map.det_data = 'Planck'


In [30]:
scan_map.apply(data)
data

TOAST INFO: Pixel data in file_c1d0s0_30GHz.FITS does not have TUNIT1 key.  Assuming 'K'.


<Data with 1 Observations:
<Observation
  name = 'POLE_DEEP-169-7'
  uid = '2008944542'  group has a single process (no MPI)
  telescope = <Telescope 'My_telescope': uid = 3533178314, site = <GroundSite 'SOUTH_POLE' : uid = 588178429, lon = -44.64999999999998 deg, lat = -89.991 deg, alt = 2842.9999999993684 m, weather = <SimWeather : 'south_pole', year = 2027, month = 4, hour = 15, site UID = 588178429, realization = 0, median = False)>, focalplane = <Focalplane: 252 detectors, sample_rate = 10.0 Hz, FOV = 29.77320077597548 deg, detectors = [029_000_SAT_f030_A .. 040_011_SAT_f030_B]>>
  session = <Session 'POLE_DEEP-169-7': uid = 1708650531, start = 2027-05-28 15:00:00+00:00, end = 2027-05-28 16:23:59.900000+00:00>
  scan_el = 54.1 deg
  scan_min_az = -1.2787014485740134 rad
  scan_max_az = 6.263463086632051 rad
  scan_min_el = 0.9442231253289323 rad
  scan_max_el = 0.9442231253289323 rad
  50400 total samples (50400 local)
  shared:  <SharedDataManager
    times (column): shape=(50400

How to center data in 0? 
Find the right average?

How does the instrument adjust between detectors? Relative average level for different detectors?

Use a 0-th order polynominial filter to remove average

In [31]:
def filter_0(obs, det_data):
    obs_arr = obs.detdata[det_data]
    obs_arr2 = np.zeros(obs_arr.shape)
    i = 0
    for detec in obs_arr:
        obs_arr2[i] = detec- np.mean(detec)
        i+=1
    obs.detdata[det_data][:,:]  = obs_arr2

In [32]:
filter_0(ob,'Planck')

For each TOD, create a corresponding Mampaker operator, in order to retrieve reconstructed maps independently.

In [33]:
binner = toast.ops.BinMap(pixel_pointing = pixels, stokes_w = weights)
mapmaker = toast.ops.MapMaker(binning = binner, template_matrix=template_matrix)

In [34]:
mapmaker.iter_max = 50
mapmaker.convergence = 1.0e-10
mapmaker.mc_mode = False

#mapmaker

#mapmaker.mc_root = 'root'

In [35]:
mapmaker.mc_mode

False

In [36]:
binner.stokes_weights = weights

In [37]:
binner.det_data = 'Planck'
mapmaker.binning= binner
mapmaker.det_data = 'Planck'
mapmaker.output_dir = './Planck'
mapmaker.apply(data)

TOAST INFO: SolveAmplitudes begin building flags for solver
TOAST INFO: SolveAmplitudes  finished flag building in 0.05 s
TOAST INFO: SolveAmplitudes begin build of solver covariance


RuntimeError: Noise model noise_model does not exist in observation POLE_DEEP-169-7